In [1]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Sequential,Model
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.regularizers import l2, l1
from keras.callbacks import ModelCheckpoint
from keras import optimizers
import numpy as np
from PIL import Image
import json

import matplotlib.pyplot as plt

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
def plot_metrics(hist, stop=50):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,4))
                            
    axes = axes.flatten()

    axes[0].plot(range(stop), hist['acc'], label='Training', color='#FF533D')
    axes[0].plot(range(stop), hist['val_acc'], label='Validation', color='#03507E')
    axes[0].set_title('Accuracy')
    axes[0].set_ylabel('Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].legend(loc='lower right')
                             
    axes[1].plot(range(stop), hist['loss'], label='Training', color='#FF533D')
    axes[1].plot(range(stop), hist['val_loss'], label='Validation', color='#03507E')
    axes[1].set_title('Loss')
    axes[1].set_ylabel('Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].legend(loc='upper right')
                             
    plt.tight_layout();

In [2]:
model = VGG16(weights='imagenet', include_top=False,  input_shape=(256, 256, 3))

In [3]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers:
    layer.trainable = False

In [4]:
x=model.output #?? quanti layer densi devo usare? e con quale numero di neuroni
x=Flatten(input_shape=model.output_shape[1:])(x)
x=Dense(256, activation='relu',  kernel_regularizer=l2(0.01))(x)
x=Dropout(0.5)(x)
preds=Dense(1, activation='sigmoid')(x) #dovrei usare softmax con due neuroni come output??

final_model=Model(inputs=model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

In [ ]:
fine_tuned_model_path = 'whole_or_demaged_model.h5'

if os.path.isfile(fine_tuned_model_path):
    final_model.load_weights(fine_tuned_model_path) #load pretrained cnn on whole-or-damaged problem
    print('weights loaded')

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255) #divide all pixel value for 225


#questo generator sembra creare immagini custom a partire dalla cartella specificata
train_generator=train_datagen.flow_from_directory('/home/denote/data/car-damage-dataset/data1a/training',
                                                 target_size=(256,256),
                                                 color_mode='rgb',
                                                 batch_size=20, #number of images for cycle
                                                 class_mode='binary',
                                                 shuffle=False)


'''A DirectoryIterator yielding tuples of (x, y) where x is a numpy array containing a batch of 
images with shape (batch_size (a.k.a. # of images), target_size, channels) 
and y is a numpy array of corresponding labels.

example of image 2x2 
[BATCH
    [   IMAGE1
        [    ROW1
            [r,g,b pixel (1,1)] , [r,g,b pixel(1,2)]
        ],
        [    ROW2
            [r,g,b pixel (1,2)] , [r,g,b pixel(2,2)]
        ]
    ],
    [   IMAGE2
        .....
    ]
]

batches[0] ->  x the numpy array 
batches[0][0] -> first image
batches[0][1] -> second image
batches[0][0][0] -> first image first row
batches[0][0][0][0] -> first image first row first pixel (rgb)
'''



Found 1840 images belonging to 2 classes.


'A DirectoryIterator yielding tuples of (x, y) where x is a numpy array containing a batch of \nimages with shape (batch_size (a.k.a. # of images), target_size, channels) \nand y is a numpy array of corresponding labels.\n\nexample of image 2x2 \n[BATCH\n    [   IMAGE1\n        [    ROW1\n            [r,g,b pixel (1,1)] , [r,g,b pixel(1,2)]\n        ],\n        [    ROW2\n            [r,g,b pixel (1,2)] , [r,g,b pixel(2,2)]\n        ]\n    ],\n    [   IMAGE2\n        .....\n    ]\n]\n\nbatches[0] ->  x the numpy array \nbatches[0][0] -> first image\nbatches[0][1] -> second image\nbatches[0][0][0] -> first image first row\nbatches[0][0][0][0] -> first image first row first pixel (rgb)\n'

In [10]:
validation_datagen = ImageDataGenerator(rescale=1./255) #divide all pixel value for 225


validation_generator = validation_datagen.flow_from_directory('/home/denote/data/car-damage-dataset/data1a/validation',
                                                           target_size=(256,256),
                                                           color_mode='rgb',
                                                           batch_size=20, #number of images for cycle
                                                           class_mode='binary',
                                                           shuffle=False)
    

Found 460 images belonging to 2 classes.


In [11]:
nb_train_samples = train_generator.n//train_generator.batch_size #number of image / batch size 
nb_validation_samples = validation_generator.n // validation_generator.batch_size

In [12]:
final_model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint(fine_tuned_model_path, monitor='val_acc', #save weights setting monitoring the best accuracy of evaluation dataset
                                 verbose=1, save_best_only=True, 
                                 save_weights_only=False, mode='auto')

fit_history = final_model.fit_generator(train_generator,
                          steps_per_epoch=nb_train_samples,
                          epochs=50,
                          validation_data=validation_generator,
                          validation_steps=nb_validation_samples,
                          verbose=1,
                          callbacks=[checkpoint])

Epoch 1/50
92/92 [==============================] - 372s 4s/step - loss: 4.6095 - acc: 0.8402 - val_loss: 4.9155 - val_acc: 0.6043

Epoch 00001: val_acc improved from -inf to 0.60435, saving model to whole_or_demaged_model.h5
Epoch 2/50
92/92 [==============================] - 371s 4s/step - loss: 4.7026 - acc: 0.7696 - val_loss: 4.8028 - val_acc: 0.6500

Epoch 00002: val_acc improved from 0.60435 to 0.65000, saving model to whole_or_demaged_model.h5
Epoch 3/50
92/92 [==============================] - 371s 4s/step - loss: 4.6663 - acc: 0.7777 - val_loss: 4.7859 - val_acc: 0.6739

Epoch 00003: val_acc improved from 0.65000 to 0.67391, saving model to whole_or_demaged_model.h5
Epoch 4/50
92/92 [==============================] - 371s 4s/step - loss: 4.6332 - acc: 0.8022 - val_loss: 4.7759 - val_acc: 0.6543

Epoch 00004: val_acc did not improve from 0.67391
Epoch 5/50
92/92 [==============================] - 371s 4s/step - loss: 4.6344 - acc: 0.7772 - val_loss: 4.7300 - val_acc: 0.6891

Ep

In [ ]:
plot_metrics(fit_history.history)